In [2]:
# Load data

import os
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 40000)
pd.set_option('display.max_rows', None)

# Read the data from the CSV file
df_safegraph_spend = pd.read_csv(os.path.join('..', 'dewey_data_filtered','SafeGraph_SPEND_Yellowstone_200miRadius.csv.gz'))

# Print the first 5 rows of the DataFrame
print(df_safegraph_spend.info())
print(df_safegraph_spend.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250070 entries, 0 to 250069
Data columns (total 36 columns):
 #   Column                                           Non-Null Count   Dtype  
---  ------                                           --------------   -----  
 0   PLACEKEY                                         250070 non-null  object 
 1   SAFEGRAPH_BRAND_IDS                              125442 non-null  object 
 2   BRANDS                                           125442 non-null  object 
 3   SPEND_DATE_RANGE_START                           250070 non-null  object 
 4   SPEND_DATE_RANGE_END                             250070 non-null  object 
 5   RAW_TOTAL_SPEND                                  250070 non-null  float64
 6   RAW_NUM_TRANSACTIONS                             250070 non-null  int64  
 7   RAW_NUM_CUSTOMERS                                250070 non-null  int64  
 8   MEDIAN_SPEND_PER_TRANSACTION                     250070 non-null  float64
 9   MEDIAN_SPEND_PE

In [16]:
# Filter by Location

# Create a geojson file using this online tool: https://geojson.io/#map=6.57/43.031/-108.404

import geopandas as gpd

filter_region = gpd.read_file(os.path.join('..', 'map','Yellowstone_100mi_Radius.geojson')) # Put your new location geojson file here

# The following code does a 2-step filtering process: 1) filter the POI down, 2) filter the SPEND data

# 1 - Filter POI within the filter_region
df_safegraph_poi = pd.read_csv(os.path.join('..', 'dewey_data_filtered','SafeGraph_POI_Yellowstone_200miRadius.csv.gz'))

print('(before filter) # of rows: ', len(df_safegraph_poi))

if 'LONGITUDE' in df_safegraph_poi.columns and 'LATITUDE' in df_safegraph_poi.columns:
    points = gpd.points_from_xy(df_safegraph_poi['LONGITUDE'], df_safegraph_poi['LATITUDE'])
    mask = points.within(filter_region.geometry.iloc[0])  # Assuming filter_region has only one polygon
    df_safegraph_poi_filtered = df_safegraph_poi[mask]
else:
    print("Error: DataFrame does not contain 'LONGITUDE' and/or 'LATITUDE' columns.")

print('(after filter)  # of rows: ', len(df_safegraph_poi_filtered))
print(df_safegraph_poi_filtered.head())

# 2 - Filter SPEND data by matching the PLACEKEY with the filtered POI
print('(before filter) # of rows: ', len(df_safegraph_spend))

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_filtered = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_safegraph_poi_filtered['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

print('(after filter)  # of rows: ', len(df_filtered))
print(df_filtered.head())


(before filter) # of rows:  75540
(after filter)  # of rows:  26790
              PLACEKEY      PARENT_PLACEKEY                                     LOCATION_NAME        SAFEGRAPH_BRAND_IDS   BRANDS  STORE_ID                                       TOP_CATEGORY                               SUB_CATEGORY  NAICS_CODE   LATITUDE   LONGITUDE          STREET_ADDRESS              CITY REGION  POSTAL_CODE ISO_COUNTRY_CODE  PHONE_NUMBER OPEN_HOURS CATEGORY_TAGS OPENED_ON CLOSED_ON TRACKING_CLOSED_SINCE GEOMETRY_TYPE               DOMAINS                      WEBSITE
2  222-222@5wf-k54-nt9                  NaN                               Headwall Sports Llc                        NaN      NaN       NaN  Sporting Goods, Hobby, and Musical Instrument ...                      Sporting Goods Stores       45111  45.263333 -111.254519       47875 Gallatin Rd  Gallatin Gateway     MT      59730.0               US           NaN        NaN           NaN       NaN       NaN               2019-07       POL

In [17]:
# Filter by Dates

print('(before filter) # of rows: ', len(df_filtered))

datetime_start = pd.to_datetime('2020-01-01')
datetime_end = pd.to_datetime('2020-03-01')

df_safegraph_spend['SPEND_DATE_RANGE_START'] = pd.to_datetime(df_safegraph_spend['SPEND_DATE_RANGE_START'])
df_safegraph_spend['SPEND_DATE_RANGE_END'] = pd.to_datetime(df_safegraph_spend['SPEND_DATE_RANGE_END'])

df_filtered = df_safegraph_spend[(df_safegraph_spend['SPEND_DATE_RANGE_START'] >= datetime_start) &
                                 (df_safegraph_spend['SPEND_DATE_RANGE_END'] <= datetime_end)]

print('(after filter) # of rows: ', len(df_filtered))
print(df_filtered.head())


(before filter) # of rows:  96013
(after filter) # of rows:  7485
                  PLACEKEY       SAFEGRAPH_BRAND_IDS            BRANDS SPEND_DATE_RANGE_START SPEND_DATE_RANGE_END  RAW_TOTAL_SPEND  RAW_NUM_TRANSACTIONS  RAW_NUM_CUSTOMERS  MEDIAN_SPEND_PER_TRANSACTION  MEDIAN_SPEND_PER_CUSTOMER SPEND_PER_TRANSACTION_PERCENTILES                                       SPEND_BY_DAY                       SPEND_PER_TRANSACTION_BY_DAY                               SPEND_BY_DAY_OF_WEEK                                         DAY_COUNTS  SPEND_PCT_CHANGE_VS_PREV_MONTH  SPEND_PCT_CHANGE_VS_PREV_YEAR  ONLINE_TRANSACTIONS  ONLINE_SPEND TRANSACTION_INTERMEDIARY SPEND_BY_TRANSACTION_INTERMEDIARY                  BUCKETED_CUSTOMER_FREQUENCY               MEAN_SPEND_PER_CUSTOMER_BY_FREQUENCY                          BUCKETED_CUSTOMER_INCOMES                  MEAN_SPEND_PER_CUSTOMER_BY_INCOME                                 CUSTOMER_HOME_CITY         RELATED_CROSS_SHOPPING_PHYSICAL_BRANDS_PCT        RE